## IPC 코드 예제

- 기본 IPC 기법은 리눅스(UNIX 계열)의 전통적인 C언어 기반에서 활용하는 것이 일반적임
- 파이썬의 경우, Redis (In-Memory 데이터베이스)의 메커니즘을 활용한 IPC 기법을 많이 활용함

### Redis PubSub 메커니즘
- 일종의 message queue 로 이해할 수 있음 (구현은 매우 다름)

### 테스트 환경
- 맥 또는 리눅스
  - 윈도우 환경의 경우, AWS EC2에서 실행 권장


- redis 설치
  - 맥
    ```
    1. brew install redis    
    2. brew services start redis
    ```

  - 리눅스
    ```
    1. sudo add-apt-repository ppa:chris-lea/redis-server
    2. sudo apt-get update
    3. sudo apt-get install redis-server
       - redis client 도 함께 설치됨
    4. sudo vi /etc/redis/redis.conf
    5. #bind 127.0.0.1 -> bind 0.0.0.0 로 변경 후 저장
    6. sudo service redis restart    
    ```

- 파이썬 redis 라이브러리 설치
  ```
  pip install redis
  ```

### 데이터 수신 코드
- 주요 코드
  - redis.Redis(): default로 자기 PC에 있는 redis에 접속함
  - pubsub(): pubsub 메커니즘을 사용하기 위한 인스턴스 생성
  - subscribe(채널명=함수명): 해당 채널명으로 데이터 수신시, 명시된 함수를 호출
  - run_in_thread(sleep_time=시간)
    - 해당 채널명의 데이터 수신을 위해, 별도 쓰레드를 생성해서, 루프를 돌며 해당 채널의 데이터를 확인하는 기능
    - sleep_time 명시를 통해서, 루프를 돌때, 일정시간 sleep 하여, 해당 채널 외에 프로세스가 처리해야하는 바를 처리함 (보통 0.001 로 넣음)

In [ ]:
import time, redis

redis_ins = redis.Redis()
pubsub_ins = redis_ins.pubsub()

def handler(message):
    print(message['data'].decode('utf-8'))
    thread.stop()

pubsub_ins.subscribe(my_channel=handler)
thread = pubsub_ins.run_in_thread(sleep_time=0.001)

### 데이터 송신 코드
- 주요 코드
  - redis.Redis(): default로 자기 PC에 있는 redis에 접속함
  - pubsub(): pubsub 메커니즘을 사용하기 위한 인스턴스 생성
  - publish(채널명, 데이터): 해당 채널명으로 데이터를 송신

In [ ]:
import redis

redis_ins = redis.Redis()
pubsub_ins = redis_ins.pubsub()

redis_ins.publish('my_channel', 'example data')

### 멀티 프로세스 예제

- 결과 수집 프로그램

In [ ]:
import time, redis

redis_ins = redis.Redis()
pubsub_ins = redis_ins.pubsub()

g_count, collect_count = 0, 0
def handler(message):
    global g_count, collect_count
    collect_count += 1
    g_count += int(message['data'])
    if collect_count == 50:
        print (g_count)
        print (time.time() - start)
        thread.stop()

pubsub_ins.subscribe(my_channel=handler)
start = time.time()
thread = pubsub_ins.run_in_thread(sleep_time=0.001)

- 결과 계산 프로그램
  - 멀티 프로세스를 사용하여, 계산량을 분리하고 병렬 처리

In [ ]:
import multiprocessing, time, os, redis

redis_ins = redis.Redis()
pubsub_ins = redis_ins.pubsub()

def process_main():
    print ('parent process:', os.getppid())
    print ('process id:', os.getpid())
    g_count = 0
    for i in range(100000):
        g_count = g_count + 1
    redis_ins.publish('my_channel', g_count)

processes = list()
print ('Main process:', os.getpid(), '\n')
for i in range(50):
    process = multiprocessing.Process(target=process_main, args=())
    processes.append(process)

for process in processes:
    process.start()
for process in processes:
    process.join()

### 참고: 파이프 예제 (네임드 파이프, Named Pipe)
- 맥/리눅스 환경에서만 지원
- OS 지원을 받아야 하므로, OS에 따라 구현이 다른 경우가 있음

In [ ]:
import os

PIPE_FILENAME = './namedpipe_test'

if not os.path.exists(PIPE_FILENAME):
    os.mkfifo(PIPE_FILENAME)

if os.path.exists(PIPE_FILENAME):
    fp_fifo = open(PIPE_FILENAME, "w")

for i in range(128):
    fp_fifo.write("Hello, I am a Process\n")
    fp_fifo.write("")

In [ ]:
import os

PIPE_FILENAME = './fifo-test'

if os.path.exists(PIPE_FILENAME):
    fp_fifo = open(PIPE_FILENAME, "r")
    i = 0
    while True:
        with open(PIPE_FILENAME, 'r') as fifo:
            data = fifo.read()
            line = data.split('\n')
            for str in line:
                i = i+1
                print(str + "%4d" % i)